In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import os
from pathlib import Path
from scipy import stats
import scipy.io
from statannot import add_stat_annotation

font = {'family' : 'Arial',
        'weight' : 'normal',
        'size'   : 8}
matplotlib.rc('font', **font)

def tolerant_mean(arrs):
    lens = [len(i) for i in arrs]
    arr = np.ma.empty((np.max(lens),len(arrs)))
    arr.mask = True
    for idx, l in enumerate(arrs):
        arr[:len(l),idx] = l
    return arr.mean(axis = -1), arr.std(axis=-1)

In [ ]:
folder = "/path/to/folder"
t_list = []
#freq_list = []
#apd90_list = []
#apd50_list = []
#apd10_list = []
trise_list = []
file_list = []
start_idx = None
stop_idx = None
for file in os.listdir(folder):
    print('processing ',file)
    file_list.append(file)
    a = scipy.io.loadmat(folder+file)
    all_data = a['AnalyzedData'][0]
    time_raw = np.ndarray.flatten(all_data[1][1])
    pk_loc = np.ndarray.flatten(all_data[6][1])
    #freq = np.ndarray.flatten(all_data[10][1])
    #apd90 = np.ndarray.flatten(all_data[12][1])
    #apd50 = np.ndarray.flatten(all_data[13][1])
    #apd10 = np.ndarray.flatten(all_data[14][1])
    trise = np.ndarray.flatten(all_data[16][1])
    time = time_raw[pk_loc][1:]
    start_filter = np.where(time>250)
    stop_filter = np.where(time<1450)
    plot_idx = np.intersect1d(start_filter, stop_filter)
    start_idx = plot_idx[0]
    stop_idx = plot_idx[-1]
    
    t_list.append(time[start_idx:stop_idx])
    #freq_list.append(freq)
    #apd90_list.append(apd90)
    #apd50_list.append(apd50)
    #apd10_list.append(apd10)
    trise_list.append(trise[start_idx:stop_idx])

In [ ]:
fig, ax = plt.subplots(figsize=(2,1.6),dpi=600)
t_list_avg = []
for i in range(len(t_list)):
    if len(t_list[i]) > len(t_list_avg):
        t_list_avg = t_list[i]
    plt.plot(t_list[i], trise_list[i], alpha=0.1, color='black', linewidth=0.6)
ax.set_ylim((0.0, 0.05))
ax.set_xlabel('time/s')
ax.set_ylabel('depolarization time/s')
#ax.set_title(folder+'n='+str(len(t_list)))
avg_trace, err = tolerant_mean(trise_list)

plt.plot(t_list_avg, avg_trace)
plt.fill_between(t_list_avg, avg_trace-err, avg_trace+err, alpha=0.3)
#despine
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.savefig(folder[:-1]+'.svg', format='svg', bbox_inches='tight')
plt.savefig(folder[:-1]+'.png', format='png', bbox_inches='tight')

In [ ]:
for t in range(len(trise_list)):
    fig, ax = plt.subplots()
    plt.plot(trise_list[t])
    ax.set_ylim((0,0.06))
    ax.set_title(file_list[t])